# Datos.

1. Nombre: Luis Felipe Narváez Gómez 
2. Código: 2312660
3. Universidad Santo Tomas seccional Tunja
4. Facultad: Ingeniería de Sistemas
5. Asignatura: Electiva I - Deep Learning
6. Periodo: 2021-2

# Deep Learning

## Introduccion a PLN Redes Neuronales Recurrentes
En la creación de redes neuronales necesitamos dos tipos de IA, para reconocer patrones o generar nuevos:

1. Las que no tienen memoria, identifica el patrón y ya!...ejemplo las de visión artificial
2. Las de memoria corta (Long Short Term Memory)...PLN
3. Las que requieren mucha memoria (aprenden casi todo...BERT)...PLN y visión artificial.


## Caso de estudio: generación de texto

Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos

Para este ejemplo usaremos “Character level language model” propuesto por Andrej Karpathy en su artículo "The Unreasonable Effectiveness of Recurrent Neural Networks"(y parcialmente basado en su implementado en el tutorial "Generate text with an RNN" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.

Tambien se recomienda ver mas informacion frente al tema:

1. https://unipython.com/generacion-de-textos-con-inteligencia-artificial/
2. https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa
3. (https://www.youtube.com/watch?v=kaQCdv46OBA&ab_channel=JeffHeaton)


# Prediciendo un texto (cuento los tres cerditos),  usando la herramienta de Keras.

## Instalar matplotlib en Anaconda3

Se debe tener encuenta en que perfil se va a instalar esta libreria de anaconda 3. Se recomienda para este caso de Uso de Tensor Flow, primero instalar y activar el mismo para instalar el resto de librerias con las que se va a trabajar:

1. conda install -c conda-forge matplotlib
2. conda install matplotlib
3. conda install matplotlib.pyplot
4. pip install matplotlib

## Importar Librerias

In [1]:
# manipulacion de archivos y carpetas
import io
import os
import requests # libreria que nos permite conectarnos a una direccion online o una URL

In [2]:
import matplotlib.pyplot as plt # Para trabajar los asuntos de generacion de graficos (una de varias librerias que hay)
plt.rcParams['figure.figsize'] = (16,9) #tamaño de la figura (ancho,alto)
plt.style.use('ggplot') # estilo grafico de las imagenes generadas
# guardar las imagenes y tablas en el notebook
%matplotlib inline 

In [3]:
import numpy as np
import time #Para uso del tiempo
import sys

In [4]:
from google.colab import drive # conexion con google Drive

ModuleNotFoundError: No module named 'google.colab'

### Instalacion de Tensor en Anaconda3

https://docs.anaconda.com/anaconda/user-guide/tasks/tensorflow/

1. Download and install Anaconda or the smaller Miniconda.

2. On Windows open the Start menu and open an Anaconda Command Prompt. On macOS or Linux open a terminal window. Use the default bash shell on macOS or Linux.

3. Choose a name for your TensorFlow environment, such as “tf”.

4. To install the current release of CPU-only TensorFlow, recommended for beginners:

In [ ]:
# conda create -n tf tensorflow
# conda activate tf

Or, to install the current release of GPU TensorFlow on Linux or Windows:

In [ ]:
#conda create -n tf-gpu tensorflow-gpu
#conda activate tf-gpu

### Matar El Ambiente de GoogleColab SI ES NECESARIO

In [ ]:
!kill -9 -1 #reiniciando la maquina virtual

### Libreria Tensor Flow

In [4]:
!pip list

Package                      Version
---------------------------- -----------
absl-py                      0.13.0
aiohttp                      3.7.4.post0
argon2-cffi                  20.1.0
astor                        0.8.1
astunparse                   1.6.3
async-generator              1.10
async-timeout                3.0.1
attrs                        21.2.0
backcall                     0.2.0
bleach                       4.0.0
blinker                      1.4
brotlipy                     0.7.0
cached-property              1.5.2
cachetools                   4.2.2
certifi                      2021.10.8
cffi                         1.15.0
chardet                      4.0.0
charset-normalizer           2.0.4
click                        8.0.3
colorama                     0.4.4
coverage                     5.5
cryptography                 3.4.8
cycler                       0.11.0
Cython                       0.29.24
debugpy                      1.4.1
decorator                    5.1.0


In [5]:
!pip install tensorflow-gpu

In [6]:
#Librerias enfocadas al Deep Learning
import tensorflow as tf

In [7]:
print("Version: ", tf.__version__)
print("GPU esta", "disponible" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print("Dispositivos disponibles: ", tf.config.list_physical_devices())

Version:  2.7.0
GPU esta disponible
Dispositivos disponibles:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### CUDA (GPU-NVIDIA)

In [8]:
print("-------------------------------RAM------------------------------")
# !cat /proc/meminfo  #cuanta memoria tenemos? # Linux Colab
!wmic MemoryChip get BankLabel, Capacity, MemoryType, TypeDetail, Speed, Manufacturer # Windows
print("-----------------------------PROCESADOR-------------------------")
# !cat /proc/cpuinfo  # que procesador tenemos?
print("-------------------------------CUDA-----------------------------")
!nvcc -V   # version de CUDA

-------------------------------RAM------------------------------
-----------------------------PROCESADOR-------------------------
-------------------------------CUDA-----------------------------


Expresi¢n GET incorrecta.



nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Wed_Jul_14_19:47:52_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.4, V11.4.100
Build cuda_11.4.r11.4/compiler.30188945_0


### Activacion de la GPU

Uso de GPU para entrenar en tensorflow

https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa

Elejir una de las siguientes dos:

In [ ]:
#tf.device('/CPU:0') #activando la CPU

In [9]:
tf.device('/GPU:0') #activando la GPU

## Conexion a Google Drive

In [ ]:
drive.mount('/content/drive')

root = '/content/drive/MyDrive/Colab Notebooks/CheckPoints/checkpoints El resplandor/Intento 1'

root = '/content/drive/MyDrive/Colab Notebooks/CheckPoints/checkpoint El Gato Negro/Intento 1'

In [29]:
# directorio donde guardar los checkpoints en google drive
root = '/content/drive/MyDrive/Colab Notebooks/CheckPoints/checkpoints El resplandor/Intento 1'

In [31]:
# directorio donde guardar los checkpoints en pc
root = 'C:/Users/ruiso/Desktop/EscritorioAnaconda3Python/Deep 2/check 1'

## DataSet

El Gato Negro:

PDF : https://github.com/OnRuiso/Deep-Learning-/blob/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_gato_negro.pdf

TXT Crudo: https://github.com/OnRuiso/Deep-Learning-/blob/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_gato_negro.txt

TXT Crudo RAW : https://raw.githubusercontent.com/OnRuiso/Deep-Learning-/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_gato_negro.txt

TXT Limpio 1: https://github.com/OnRuiso/Deep-Learning-/blob/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_gato_negro_2.txt

TXT Limpio 1 RAW: https://raw.githubusercontent.com/OnRuiso/Deep-Learning-/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_gato_negro_2.txt


El Resplandor:

PDF: https://github.com/OnRuiso/Deep-Learning-/blob/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_resplandor.pdf

TXT Crudo: https://github.com/OnRuiso/Deep-Learning-/blob/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_resplandor.txt

TXT Crudo RAW: https://raw.githubusercontent.com/OnRuiso/Deep-Learning-/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_resplandor.txt

TXT 1/2 Limpiado: https://github.com/OnRuiso/Deep-Learning-/blob/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_resplandor_2.txt

TXT 1/2 Limpio RAW: https://raw.githubusercontent.com/OnRuiso/Deep-Learning-/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_resplandor_2.txt

### Descargar y preprocesar los datos

In [10]:
fileDL= tf.keras.utils.get_file('el_resplandor_2.txt','https://raw.githubusercontent.com/OnRuiso/Deep-Learning-/main/Codigos/Deep_Learnig_IA_Data/DataSets_P1T16/el_resplandor_2.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')

In [11]:
# Si se quiere mostrar el texto que ha sido cargado
print(texto)

El Resplandor
STEPHEN KING
PRIMERA PARTE
PRELIMINARES

1. ENTREVISTA DE TRABAJO

Qué empleaducho engreído, pensó Jack Torrance.
Ullman no pasaría de un metro sesenta y cinco, y al moverse lo hacía con la melindrosa rapidez que parece ser  especialidad  exclusiva  de  los  hombres bajos y regordetes. La raya del pelo era milimétrica, y el traje oscuro, sobrio, pero reconfortante.  Un traje que  parecía  invitar  a  las  confidencias  cuando  se trataba de un cliente cumplidor, y que transmitía, en cambio, un mensaje más lacónico al ayudante contratado: más vale que sea usted eficiente.
Llevaba  un  clavel  rojo  en  la  solapa,  probablemente  para  que  por  la  calle nadie confundiera a Stuart Ullman con el empresario de pompas fúnebres.
Mientras  lo  oía  hablar,  Jack  admitió  para  sus  adentros  que,  muy probablemente,  en  esas  circunstancias  no  le  habría  gustado  a  nadie  que estuviera al otro lado del mostrador.
Ullman  le  había  hecho  una  pregunta,  sin  que  él  al

### Limpiar el texto

In [12]:
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto esta compuesto de estos :104 caracteres
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', 'º', '»', '¿', 'Á', 'É', 'Í', 'Ñ', 'Ó', 'Ú', 'á', 'ä', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '—', '“', '”', '…']


### Pasar texto a Minisculas

In [13]:
texto = texto.lower()

#### Reemplazar caracteres por espacios

In [14]:
texto = texto.replace('á','a')
texto = texto.replace('é','e')
texto = texto.replace('í','i')
texto = texto.replace('ó','o')
texto = texto.replace('ú','u')

In [15]:
texto = texto.replace('(','')
texto = texto.replace(')','')
texto = texto.replace('*','')
texto = texto.replace('!','')
texto = texto.replace('¡','')
texto = texto.replace('"','')
texto = texto.replace('&','')
texto = texto.replace("'","")
texto = texto.replace(',','')
texto = texto.replace('.','')
texto = texto.replace('-','')
texto = texto.replace('_','')
texto = texto.replace(':','')
texto = texto.replace('/','')
texto = texto.replace('$','')
texto = texto.replace('%','')
texto = texto.replace('=','')
texto = texto.replace(';','')
texto = texto.replace('?','')
texto = texto.replace('¿','')
texto = texto.replace('[','')
texto = texto.replace(']','')
texto = texto.replace('«','')
texto = texto.replace('º','')
texto = texto.replace('»','')
texto = texto.replace('“','')
texto = texto.replace('”','')
texto = texto.replace('…','')

In [16]:
texto = texto.replace('—','')
texto = texto.replace('ü','u')
texto = texto.replace('ä','a')
texto = texto.replace('\n',' ')

In [17]:
texto = texto.replace('0','')
texto = texto.replace('1','')
texto = texto.replace('2','')
texto = texto.replace('3','')
texto = texto.replace('4','')
texto = texto.replace('5','')
texto = texto.replace('6','')
texto = texto.replace('7','')
texto = texto.replace('8','')
texto = texto.replace('9','')

In [18]:
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto esta compuesto de estos :28 caracteres
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ñ']


## Entendiendo El Texto

In [19]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:1031722 caracteres
el texto esta compuesto de estos :28 caracteres
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ñ']


In [20]:
print(texto)

el resplandor stephen king primera parte preliminares   entrevista de trabajo  que empleaducho engreido penso jack torrance ullman no pasaria de un metro sesenta y cinco y al moverse lo hacia con la melindrosa rapidez que parece ser  especialidad  exclusiva  de  los  hombres bajos y regordetes la raya del pelo era milimetrica y el traje oscuro sobrio pero reconfortante  un traje que  parecia  invitar  a  las  confidencias  cuando  se trataba de un cliente cumplidor y que transmitia en cambio un mensaje mas laconico al ayudante contratado mas vale que sea usted eficiente llevaba  un  clavel  rojo  en  la  solapa  probablemente  para  que  por  la  calle nadie confundiera a stuart ullman con el empresario de pompas funebres mientras  lo  oia  hablar  jack  admitio  para  sus  adentros  que  muy probablemente  en  esas  circunstancias  no  le  habria  gustado  a  nadie  que estuviera al otro lado del mostrador ullman  le  habia  hecho  una  pregunta  sin  que  el  alcanzara  a  oirla  mal

## Pasar el texto a numeros

Las redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [ ]:
# ordena los caracteres presentes en el texto de forma alfabetica como en una lista
# a cada uno de los caracteres listados asigname un numero ordenado

In [21]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'el resplandor stephen king primera parte preliminares   entrevista de trabajo  que empleaducho engre'
array([ 5, 12,  0, 18,  5, 19, 16, 12,  1, 14,  4, 15, 18,  0, 19, 20,  5,
       16,  8,  5, 14,  0, 11,  9, 14,  7,  0, 16, 18,  9, 13,  5, 18,  1,
        0, 16,  1, 18, 20,  5,  0, 16, 18,  5, 12,  9, 13,  9, 14,  1, 18,
        5, 19,  0,  0,  0,  5, 14, 20, 18,  5, 22,  9, 19, 20,  1,  0,  4,
        5,  0, 20, 18,  1,  2,  1, 10, 15,  0,  0, 17, 21,  5,  0,  5, 13,
       16, 12,  5,  1,  4, 21,  3,  8, 15,  0,  5, 14,  7, 18,  5])


In [ ]:
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

### Exportar Vocablos y matriz numerica

## Preparar los datos para ser usados en la RNN

In [22]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=50
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'el resplandor stephen king primera parte preliminar'
'es   entrevista de trabajo  que empleaducho engreid'
'o penso jack torrance ullman no pasaria de un metro'
' sesenta y cinco y al moverse lo hacia con la melin'
'drosa rapidez que parece ser  especialidad  exclusi'
'va  de  los  hombres bajos y regordetes la raya del'
' pelo era milimetrica y el traje oscuro sobrio pero'
' reconfortante  un traje que  parecia  invitar  a  '
'las  confidencias  cuando  se trataba de un cliente'
' cumplidor y que transmitia en cambio un mensaje ma'


### Separar los datos en agrupamientos (batches)

In [23]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)

for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'el resplandor stephen king primera parte prelimina'
Target data:  'l resplandor stephen king primera parte preliminar'


In [ ]:
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )

In [24]:
#imprimimos el tensor del dataset
print(dataset)

<MapDataset shapes: ((50,), (50,)), types: (tf.int32, tf.int32)>


In [25]:
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000

In [26]:
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)

<BatchDataset shapes: ((64, 50), (64, 50)), types: (tf.int32, tf.int32)>


In [ ]:
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

## Construcción del modelo RNN

Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.

In [27]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           7168      
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 28)            28700     
                                                                 
Total params: 5,282,844
Trainable params: 5,282,844
Non-trainable params: 0
_________________________________________________________________


## Creando chekpoints

Una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

In [32]:
print(root)

C:/Users/ruiso/Desktop/EscritorioAnaconda3Python/Deep 2/check 1


In [33]:
checkpoint_dir=root

Se utiliza un checkpoint_dir en un espacio en DRIVE para mejorar la situacion del entranimiento. Google Colab ofrece un espacio en disco limitado para guardar variables, cada checkpoint en este proceso en concreto puede llegar a pesar 60 a 70MB, con un entrenamiento relativamente basico de 1000 Epochs, tendriamos un espacio de 70000MB o 70Gb en disco de los cuales requerir.

In [34]:
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')

## Entrenando la RNN

### Entrenamiento desde Checkpoint 0

Se recomienda investigar los siguientes enlaces de informacion:

1. https://keras.io/api/callbacks/model_checkpoint/
2. https://towardsdatascience.com/checkpointing-deep-learning-models-in-keras-a652570b8de6

In [35]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Wed_Jul_14_19:47:52_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.4, V11.4.100
Build cuda_11.4.r11.4/compiler.30188945_0


In [39]:
# ciclos de entrenamiento = 500
# verbose 0 = no quiero ver por terminal la descripcion de cada ciclo de entrenamiento
# vervose 0 = menor requerimiento de recursos en el entrenamiento, se utiliza netamiente la GPU para entrenar

EPOCHS=500
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=0,
                  callbacks=[cp_callback])

KeyboardInterrupt: 

Tambien se puede entranar atravez de otro checkpoint.

Desde la carpeta que optamos guardar los checkpoints

1. el archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
2. el archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados

In [40]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

C:/Users/ruiso/Desktop/EscritorioAnaconda3Python/Deep 2/check 1\cp_0059.ckpt


## Generando Texto usando RNN

In [41]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [42]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

### Generando texto

In [43]:
print(generate_text(model, start_string=u"tienes que matarlo"))

tienes que matarlo  si papa pero wendy  si  señor  aparecio  la  idea  a  parar  al consultorio  en  el  cuarto  de  baño  y  cerro  lentamente  la  cabeza  en  un gesto de sus labios  sosteniendo  la  cabeza  perfectamente  inmovil  podia  ver  el  minutero  arriba  en  el  suelo  con  un  resplandor  de  luces florida  como  una  pata  y  se  sintio  aliviada  por  la  noche  como  si  le  hubieran  quitado  un peso de los cables  danny empezo a lloriquear  se  detuvo  wendy  oyo  el  silbido  del  bolsillo  de


## Exportando modelo

Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).

In [44]:
from keras.models import model_from_json
import os

In [45]:
dir_export= 'C:/Users/ruiso/Desktop/EscritorioAnaconda3Python/Deep 2/export 1'

In [46]:
print(dir_export)

C:/Users/ruiso/Desktop/EscritorioAnaconda3Python/Deep 2/export 1


In [47]:
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_LasMinasDelReySalomon_json.json'), 'w') as json_file:
    json_file.write(model_json)

    
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_ElResplandor_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_ElResplandor_model.h5'))
print("modelo salvado en Directorio Local")

modelo salvado en Directorio Local


## Cargando modelo serializado

### Descargar el modelo usando PYRIND & URLLIB

In [49]:
!pip install pyprind

In [50]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


In [51]:
import urllib.request
url_github_Model='https://github.com/OnRuiso/Deep-Learning-/blob/main/Codigos/Deep_Learnig_IA_Data/CheckPoint%20Data/P1T17%20El%20Resplandor/export%201/RNN_ElResplandor_model.h5'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_ElResplandor_model.h5', 
                           reporthook)

-17203200% | 0 MB | 0.46 MB/s | 0 segundos transcurrido

('RNN_ElResplandor_model.h5', <http.client.HTTPMessage at 0x25f6b999100>)

In [60]:
new_model = tf.keras.models.load_model('C:/Users/ruiso/Desktop/EscritorioAnaconda3Python/Deep 2/RNN_ElResplandor_model.h5')

OSError: SavedModel file does not exist at: C:/Users/ruiso/Desktop/EscritorioAnaconda3Python/Deep 2/RNN_ElResplandor_model.h5\{saved_model.pbtxt|saved_model.pb}

In [ ]:
print(generate_text(model, start_string=u"tienes que matarlo"))